<html>
    <h1>  AdaBoost 
        </h1>
        <p> <b> Min Khant Soe</b> </p>
        <p> ID: 122277 </p>
        <p> ^_^ </p>
    </html>

Modify the AdaBoost scratch code in our lecture such that:

Notice that if err = 0, then  will be undefined, thus attempt to fix this by adding some very small value to the lower term

Notice that sklearn version of AdaBoost has a parameter learning_rate. This is in fact the  in front of the  calculation.
Attempt to change this  into a parameter called eta, and try different values of it and see whether accuracy is improved. Note that sklearn default this value to 1.

Observe that we are actually using sklearn DecisionTreeClassifier. If we take a look at it closely, it is actually using weighted gini index, instead of weighted errors that we learn above. Attempt to write your own class of class Stump that actually uses weighted errors, instead of weighted gini index. 

To check whether your stump really works, it should give you still relatively the same accuracy. In addition, if you do not change y to -1, it will result in very bad accuracy. Unlike sklearn version of DecisionTree, it will STILL work even y is not change to -1 since it uses gini index

Put everything into a class

In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import classification_report

X, y = make_classification(n_samples=500, random_state=10)
y = np.where(y==0,-1,1)  #change our y to be -1 if it is 0, otherwise 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=35)

In [2]:
print(X.shape,y.shape,"\n",X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(500, 20) (500,) 
 (300, 20) (200, 20) (300,) (200,)


In [3]:
class Stump():
    def __init__(self):
        # Determines whether threshold should be evaluated as < or >
        self.polarity = 1
        self.n_index = None
        self.threshold = None
        # Voting power of the stump
        self.alpha = None

In [4]:
class Ada_Boost():
    def __init__(self, S=5, eta=0.5):
        self.S = S
        self.eta = eta
        
    def fit(self, X, y): #<----X_train, y_train
        m, n = X.shape
        
        #initial weight
        W = np.full(m, 1/m)
                
        self.clfs = []   #Classifier
        
        for _ in range(self.S):
            clf = Stump()
            
            #set initial minimum error to infinity
            min_err = np.inf

            for n in range(n):
                n_val = np.sort(np.unique(X[:, n]))
                thresholds = (n_val[:-1] + n_val[1:])/2
                
                for threshold in thresholds:
                    for polarity in [1, -1]:
                        y_hat = np.ones(len(y)) #set all to 1
                        y_hat[polarity * X[:, n] < polarity * threshold] = -1  #polarity = 1 rule
                        err = W[(y_hat != y)].sum()
                                        
                        #To save the best stump
                        if err < min_err:
                            clf.polarity = polarity
                            clf.threshold = threshold
                            clf.n_index = n
                            min_err = err
        
            #After the best stump is found
            #Calculate alpha, and reweight samples
            eps = 0.000001             #to prevent division by zero
            clf.alpha = self.eta * (np.log ((1 - min_err) / (min_err + eps)))            
            W = W * np.exp(-clf.alpha * y * y_hat)
            W = W / sum (W)

            #save clf
            self.clfs.append(clf)
        
    def predict(self, X):
        m, n = X.shape
        y_hat = np.zeros(m)
        for clf in self.clfs:
            pred = np.ones(m) #set all to 1
            pred[clf.polarity * X[:, clf.n_index] < clf.polarity * clf.threshold] = -1  #polarity = 1 rule
            y_hat += clf.alpha * pred

        return np.sign(y_hat)

In [5]:
model = Ada_Boost(S=20, eta = 1/2)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.49      0.99      0.66        96
           1       0.83      0.05      0.09       104

    accuracy                           0.50       200
   macro avg       0.66      0.52      0.37       200
weighted avg       0.67      0.50      0.36       200



In [6]:
model = Ada_Boost(S=20, eta = 1)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.49      1.00      0.66        96
           1       1.00      0.03      0.06       104

    accuracy                           0.49       200
   macro avg       0.74      0.51      0.36       200
weighted avg       0.75      0.49      0.34       200



In [7]:
model = Ada_Boost(S=20, eta = 1/4)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.49      0.99      0.66        96
           1       0.83      0.05      0.09       104

    accuracy                           0.50       200
   macro avg       0.66      0.52      0.37       200
weighted avg       0.67      0.50      0.36       200



In [8]:
model = Ada_Boost(S = 15, eta = 1/2)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.49      0.99      0.66        96
           1       0.86      0.06      0.11       104

    accuracy                           0.51       200
   macro avg       0.67      0.52      0.38       200
weighted avg       0.68      0.51      0.37       200



In [9]:
model = Ada_Boost(S = 5, eta = 1/2)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.71      0.98      0.82        96
           1       0.97      0.63      0.77       104

    accuracy                           0.80       200
   macro avg       0.84      0.81      0.80       200
weighted avg       0.85      0.80      0.79       200



In [10]:
model = Ada_Boost(S = 5, eta = 1/10000000)
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(classification_report(y_test, yhat))

              precision    recall  f1-score   support

          -1       0.87      0.95      0.91        96
           1       0.95      0.87      0.90       104

    accuracy                           0.91       200
   macro avg       0.91      0.91      0.90       200
weighted avg       0.91      0.91      0.90       200



Notice that sklearn version of AdaBoost has a parameter learning_rate. This is in fact the $\frac{1}{2}$ in front of the $\alpha$ calculation. Attempt to change this $\frac{1}{2}$ into a parameter called eta, and try different values of it and see whether accuracy is improved. Note that sklearn default this value to 1.

________

Accuracy can be improved by decreasing eta with same S or decreasing S value with same eta.

The maximum accuracy I could find from this is 91%.